# MultiModal Kinetic Modeling

In this tutorial we will investigate how to simultaneously optimize transient optical and transient X-ray emission spectroscopy 

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)
%matplotlib qt

## 1) Load Data


In [ ]:
filepath = os.path.join(os.getcwd(), 'Data', 'MultiModal')  # set path to file to fit

Prepare the transient optical data

In [ ]:
ta=pf.TA(filename='combined_optical_spectrum.SIA',path=filepath)  #load transient optical
plt.close('all')
#ta.Background()
ta.wavelength_bin=20                                        # width of wavelength bin in kinetics
ta.wave_nm_bin=5;
ta.ignore_time_region
ta.rel_wave=np.concatenate([np.arange(300,572+25,25),np.arange(662,800,25)])# rebinning wavelength
ta.timelimits=[-0.2,100]                                     # use to manually limit the range to plot (and fit)     
ta.bordercut=[335,766]                                      # use to set the outer wavelength limits
ta.scattercut=[572,662]                                     # this region is blanked out to block scatter, this can be a list of regions
ta.intensity_range=1.2e-2                          # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
ta.Plot_RAW([0,3])                                      # Plot Raw Spectra

In [ ]:
import function_library as func

In [ ]:
plt.close('all')                                             # Close old spectra
ta.mod=func.P12                                        # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.275,vary=True)                                # Add second rate component
par.add('k1',value=1/7.2,vary=True)                                # Add one rate component
par.add('k2',value=1/3,vary=True)                                 # Add second rate component
par.add('t0',value=-0.0167,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.0935,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
#par.add('background')                                              # Keyword for an non decaying component                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.ignore_time_region=[-0.5,0.3]                                      # select time window in which to fit
ta.log_fit=False                                               # fit in linear or log scale
ta.timelimits=[-1,200]
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted       
ta.log_scale=False
ta.Plot_fit_output([0,4])# 
#ta.Plot_fit_output(range(5))                                 # plot the fit output

In [ ]:
ta.Save_Powerpoint()

These results are terrible, mainly because the optical artifacts in the data are disturbing the system. Lets see what the XES is doing.

In [ ]:
xes=pf.TA('XES_on.SIA',path=filepath,sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission intensity',baseunit='ps')   #
plt.close('all')
xes.ds=pf.Frame_golay(xes.ds,7,1)
xes.rel_wave=np.arange(7053,7061,1)      # use to manually select interesting wavelength for the kinetics
xes.wavelength_bin=1                                        # width of wavelength bin in kinetics
xes.rel_time=[-0.5,-0.1,0,0.1,0.3,0.5,1,1.5,3,5,10,15,20]      # use to manually set interesting times for the spectra
xes.time_width_percent=0                                    # rebinning of time_points in percent
xes.timelimits=[-1,200]                                     # use to manually limit the range to plot (and fit)     
xes.log_scale=False                                         # use to plot the 2d plots with logarithmic intensity scale
xes.bordercut=[7045,7065]                                      # use to set the outer wavelength limits
xes.intensity_range=[0,7]                           # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
#xes.Plot_RAW()                                      # Plot Raw Spectra

In [ ]:
plt.close('all')                                            
xes.mod=func.P12                                       
par=ta.par_fit                                       
par['t0'].value=-0.0167                               
par['resolution'].value=0.0935
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect

if 0:
    for key in par.keys():
        par[key].vary=False
xes.par=par                                                     # write parameter object into file for fitting
xes.Fit_Global()                                 # trigger fitting
xes.error_matrix_amplification=1;                              # Choose how much stronger should error be plotted                                                    # 2D plots in linear or log scale
xes.Plot_fit_output([0,4])                                 # plot the fit output

In [ ]:
xes.Save_Powerpoint()

In [ ]:
xes.Background(uplimit=-0.25)
xes.intensity_range=1
try:
    xes.par.pop('background')
except:
    pass
xes.Fit_Global(confidence_level=0.95)                                 # trigger fitting
plt.close('all')
xes.Plot_fit_output([0,4])

In [ ]:
xes.Save_Powerpoint()

In [ ]:
weights=[xes.re['A'].abs().sum().sum(),ta.re['A'].abs().sum().sum()]
print(weights)
ta1=ta.Copy()
xes1=xes.Copy()

In [ ]:
for key in ['k0','k1','k2']:
	xes1.par[key].vary=True

In [ ]:
xes1.Fit_Global(multi_project=[ta1],same_DAS=False,same_shape_params=False,weights=weights,unique_parameter=['resolution','t0'],confidence_level=0.95)  # trigger fitting
xes1.Plot_fit_output()

In [ ]:
xes2=xes.Copy()
weights=[1/xes.re['A'].abs().sum().sum(),1/ta.re['A'].abs().sum().sum()]
xes2.Fit_Global(multi_project=[ta1],same_DAS=False,same_shape_params=False,weights=weights,unique_parameter=['resolution','t0'],confidence_level=0.95)  # trigger fitting
xes2.Plot_fit_output()

In [ ]:
xes2.filename='XES_with_TA_refined'
xes2.Save_Powerpoint()

In [ ]:
xes1.par=ta1.par_fit
for key in xes1.par.keys():
    xes1.par[key].vary=False
xes1.Fit_Global()
plt.close('all')
xes1.Plot_fit_output()